<a href="https://colab.research.google.com/github/asushalaev/DataSciencePortfolio/blob/main/DataSciencePortfolioProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ingest


## Exploratory Data Analisys

## Data Modeling